In [29]:
import pandas as pd
import numpy as np
df=pd.read_excel("Final_Neonatal data.xlsx")

In [5]:
df.shape

(3009, 44)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [8]:
drop_cols = [
    'CardNumber', 'FullName', 'Region', 'Zone', 'Wereda', 'Kebele',
    'APGARscore','Xray', 'US', 'EchoCard', 'CTScan', 'Hepatatis',
    'Unnamed: 44', '1'
]

# Filter out columns that do not exist in the DataFrame
existing_drop_cols = [col for col in drop_cols if col in df.columns]

# Drop only the existing columns
d = df.drop(columns=existing_drop_cols)

print(f"Dropped columns: {existing_drop_cols}")
print("Remaining columns:")
print(d.columns.tolist())

Dropped columns: ['CardNumber', 'FullName', 'Region', 'Zone', 'Wereda', 'Kebele', 'APGARscore', 'Xray', 'US', 'EchoCard', 'CTScan', 'Hepatatis']
Remaining columns:
['RegistrationDate', 'Sex', 'AgeInDay', 'AgeInHour', 'Wight', 'Diagnosis', 'TotalWBC', 'Hemoglobin', 'Hematocrit', 'Platelet', 'BloodRH1', 'RBS', 'ESR', 'CRP', 'Direct', 'Indirect', 'Total', 'Na', 'Ca', 'K', 'Mg', 'ManagmentDecision', 'AgeMother', 'Gravidity', 'Parity', 'BloodRH2', 'HIV', 'GestesionalAge', 'PlaceofDeli', 'UDRL', 'DischargeDate', 'DischargeCondition']


In [9]:
d.to_excel('Final_Neonatal datanew.xlsx', index=False)

In [10]:
d=pd.read_excel('Final_Neonatal datanew.xlsx')

In [12]:
d.shape

(3009, 32)

In [13]:
# Check how many samples per class
print(d['DischargeCondition'].value_counts())

DischargeCondition
Improved    2501
Died         368
LAMA         108
Transfer      32
Name: count, dtype: int64


In [14]:
d.drop_duplicates(inplace=True)

In [15]:
print(d['DischargeCondition'].value_counts())

DischargeCondition
Improved    977
Died        149
LAMA         42
Transfer     12
Name: count, dtype: int64


In [ ]:
d.isnull().sum()

In [20]:
mode_sex = d['Sex'].mode()[0]
d['Sex'] = d['Sex'].fillna(mode_sex)

In [21]:
mode_diagnosis = d['Diagnosis'].mode()[0]
d['Diagnosis'] = d['Diagnosis'].fillna(mode_diagnosis)

In [22]:
mode_ManagmentDecision = d['ManagmentDecision'].mode()[0]
d['ManagmentDecision'] = d['ManagmentDecision'].fillna(mode_ManagmentDecision)

In [23]:
mode_ManagmentDecision = d['ManagmentDecision'].mode()[0]
d['ManagmentDecision'] = d['ManagmentDecision'].fillna(mode_ManagmentDecision)

In [24]:
mean_GestesionalAge = d['GestesionalAge'].mean()
d['GestesionalAge'] = d['GestesionalAge'].fillna(mean_GestesionalAge)

In [26]:
mean_udrl = d['UDRL'].mean()
d['UDRL'] = d['UDRL'].fillna(mean_udrl)

In [ ]:
d.isnull().sum()

In [30]:


def clinical_entropy_matched_oversample(
    d,
    target_col,
    total_size=None,
    target_entropy=1.23,
    noise_level=0.02,
    random_state=42
):

    d = d.copy()
    numeric_cols = d.select_dtypes(include=[np.number]).columns.tolist()
    if target_col in numeric_cols:
        numeric_cols.remove(target_col)  # avoid adding noise to target

    counts = d[target_col].value_counts()
    classes = counts.index.tolist()
    freqs = counts.values.astype(float)

    log_freqs = np.log(freqs)

    def entropy(p):
        return -np.sum(p * np.log(p + 1e-9))

    if total_size is None:
        total_size = len(df)

    if target_entropy is None:
        # maximum entropy for current number of classes
        target_entropy = np.log(len(classes))

    # Search temperature
    best_T, best_diff, best_p = None, 1e9, None
    for T in np.linspace(0.1, 5, 5000):
        logits = log_freqs / T
        p = np.exp(logits) / np.exp(logits).sum()
        diff = abs(entropy(p) - target_entropy)
        if diff < best_diff:
            best_T, best_diff, best_p = T, diff, p

    # Final target counts
    targets = dict(zip(classes, (best_p * total_size).astype(int)))

    augmented = [d]

    for cls, target_n in targets.items():
        class_df = df[df[target_col] == cls]
        n_current = len(class_df)

        if target_n <= n_current:
            continue

        n_needed = target_n - n_current
        sampled = class_df.sample(n=n_needed, replace=True, random_state=random_state).copy()

        # Add noise to numeric columns
        for col in numeric_cols:
            std = d[col].std()
            if std > 0:
                sampled[col] += np.random.normal(0, noise_level * std, size=n_needed)

        # Optional: cast integers back if original column is int
        for col in numeric_cols:
            if pd.api.types.is_integer_dtype(d[col]):
                sampled[col] = sampled[col].round().astype(df[col].dtype)

        augmented.append(sampled)

    df_aug = pd.concat(augmented, ignore_index=True)
    df_aug = df_aug.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return df_aug


In [33]:
df_aug = clinical_entropy_matched_oversample(
    d,
    target_col='DischargeCondition',
    total_size=2065,
    target_entropy=1.23,
    noise_level=0.02
)

df_aug.to_excel('Final_Neonatal_data_augmentedml.xlsx', index=False)


In [34]:
df_aug

,RegistrationDate,Sex,AgeInDay,AgeInHour,Wight,Diagnosis,TotalWBC,Hemoglobin,Hematocrit,Platelet,...,Region,Zone,Wereda,Kebele,APGARscore,Xray,US,EchoCard,CTScan,Hepatatis
0,2015-04-04 00:00:00,Male,4.000000,0.000000,2.600000,"Jaundice,Hyphothermia,SGH",10.000000,17.000000,50.000000,230.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-02-22 00:00:00,Male,28.061599,0.003002,3.182427,"Sepsis,Anemia,Hyphoglicomia,",31467.850080,9.304017,24.116072,393.265239,...,Amhara,South Wollo,worebabo,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-10-12 00:00:00,Male,2.000000,0.000000,2.800000,"Sepsis,Jaundice,",14.800000,12.600000,42.400000,259.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-05-18 00:00:00,Male,2.000000,0.000000,1.800000,"Sepsis,",27.300000,13.600000,41.500000,251.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-08-03 00:00:00,Female,1.000000,0.000000,2.000000,"Low Birth Wight (LBW),Sepsis,",9000.000000,17.800000,57.700000,136.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,2015-06-01 00:00:00,Male,1.000000,0.000000,3.200000,"Sepsis,un able to suck",21.800000,12.700000,38.400000,165.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1736,2015-02-28 00:00:00,Female,0.968440,-0.007773,1.384986,"Low Birth Wight (LBW),Sepsis,RDS,Hyphothermia,",7176.149261,17.130068,47.461405,209.857000,...,Amhara,South Wollo,d/zuria,38.0,NaN,NaN,NaN,NaN,NaN,NaN
1737,2015-08-07 00:00:00,Male,1.000000,0.000000,3.400000,"MMC,Hyphothermia,",11.900000,15.000000,45.000000,249.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1738,2015-02-17 00:00:00,Female,17.942560,0.003619,2.258197,"Low Birth Wight (LBW),Sepsis,b.meningitis, CHF...",7754.236265,17.824723,55.657121,54.288755,...,Amhara,South Wollo,woreilu,4.0,NaN,chest x-ray=left diamatric hernia=pneumonia,abdominopulvic u/s=normal finding,normal echo study,NaN,NaN
